# Enefit - Predict Energy Behavior of Prosumers
<h4>Predict Prosumer Energy Patterns and Minimize Imbalance Costs:</h4>
<p>Your challenge in this competition is to predict the amount of electricity produced and consumed by Estonian energy customers who have installed solar panels. You'll have access to weather data, the relevant energy prices, and records of the installed photovoltaic capacity.</p>

In [1]:
import os, sys
import datetime
import holidays
import json

import numpy as np
import pandas as pd
import polars as pl


import gc
import pickle

import joblib
from joblib import load
from sklearn.ensemble import VotingRegressor
# import lightgbm as lgb



import warnings
warnings.filterwarnings("ignore")


In [2]:
# For Local
import public_timeseries_testing_util as enefit

# # For Live
# import enefit

### Data Classes

In [27]:
class DataStorage:
    # root = "/kaggle/input/predict-energy-behavior-of-prosumers"
    root = os.getcwd()

    data_cols = [
        "target",
        "county",
        "is_business",
        "product_type",
        "is_consumption",
        "datetime",
        "row_id",
    ]
    client_cols = [
        "product_type",
        "county",
        "eic_count",
        "installed_capacity",
        "is_business",
        "date",
    ]
    gas_prices_cols = ["forecast_date", "lowest_price_per_mwh", "highest_price_per_mwh"]
    electricity_prices_cols = ["forecast_date", "euros_per_mwh"]
    forecast_weather_cols = [
        "latitude",
        "longitude",
        "hours_ahead",
        "temperature",
        "dewpoint",
        "cloudcover_high",
        "cloudcover_low",
        "cloudcover_mid",
        "cloudcover_total",
        "10_metre_u_wind_component",
        "10_metre_v_wind_component",
        "forecast_datetime",
        "direct_solar_radiation",
        "surface_solar_radiation_downwards",
        "snowfall",
        "total_precipitation",
    ]
    historical_weather_cols = [
        "datetime",
        "temperature",
        "dewpoint",
        "rain",
        "snowfall",
        "surface_pressure",
        "cloudcover_total",
        "cloudcover_low",
        "cloudcover_mid",
        "cloudcover_high",
        "windspeed_10m",
        "winddirection_10m",
        "shortwave_radiation",
        "direct_solar_radiation",
        "diffuse_radiation",
        "latitude",
        "longitude",
    ]
    location_cols = ["longitude", "latitude", "county"]
    target_cols = [
        "target",
        "county",
        "is_business",
        "product_type",
        "is_consumption",
        "datetime",
    ]

    def __init__(self):
        self.df_data = pl.read_csv(
            os.path.join(self.root, "train.csv"),
            columns=self.data_cols,
            try_parse_dates=True,
        )
        self.df_client = pl.read_csv(
            os.path.join(self.root, "client.csv"),
            columns=self.client_cols,
            try_parse_dates=True,
        )
        self.df_gas_prices = pl.read_csv(
            os.path.join(self.root, "gas_prices.csv"),
            columns=self.gas_prices_cols,
            try_parse_dates=True,
        )
        self.df_electricity_prices = pl.read_csv(
            os.path.join(self.root, "electricity_prices.csv"),
            columns=self.electricity_prices_cols,
            try_parse_dates=True,
        )
        self.df_electricity_prices = self.df_electricity_prices.drop_nulls()
        
        self.df_forecast_weather = pl.read_csv(
            os.path.join(self.root, "forecast_weather.csv"),
            columns=self.forecast_weather_cols,
            try_parse_dates=True,
        )
        self.df_historical_weather = pl.read_csv(
            os.path.join(self.root, "historical_weather.csv"),
            columns=self.historical_weather_cols,
            try_parse_dates=True,
        )
        self.df_weather_station_to_county_mapping = pl.read_csv(
            os.path.join(self.root, "weather_station_to_county_mapping.csv"),
            columns=self.location_cols,
            try_parse_dates=True,
        )
        self.df_data = self.df_data.filter(
            pl.col("datetime") >= pd.to_datetime("2022-01-01")
        )
        self.df_target = self.df_data.select(self.target_cols)

        self.schema_data = self.df_data.schema
        self.schema_client = self.df_client.schema
        self.schema_gas_prices = self.df_gas_prices.schema
        self.schema_electricity_prices = self.df_electricity_prices.schema
        self.schema_forecast_weather = self.df_forecast_weather.schema
        self.schema_historical_weather = self.df_historical_weather.schema
        self.schema_target = self.df_target.schema

        self.df_weather_station_to_county_mapping = (
            self.df_weather_station_to_county_mapping.with_columns(
                pl.col("latitude").cast(pl.datatypes.Float32),
                pl.col("longitude").cast(pl.datatypes.Float32),
            )
        )

    def update_with_new_data(
        self,
        df_new_client,
        df_new_gas_prices,
        df_new_electricity_prices,
        df_new_forecast_weather,
        df_new_historical_weather,
        df_new_target,
    ):
        df_new_client = pl.from_pandas(
            df_new_client[self.client_cols], schema_overrides=self.schema_client
        )
        df_new_gas_prices = pl.from_pandas(
            df_new_gas_prices[self.gas_prices_cols],
            schema_overrides=self.schema_gas_prices,
        )
        df_new_electricity_prices = pl.from_pandas(
            df_new_electricity_prices[self.electricity_prices_cols],
            schema_overrides=self.schema_electricity_prices,
        )
        df_new_forecast_weather = pl.from_pandas(
            df_new_forecast_weather[self.forecast_weather_cols],
            schema_overrides=self.schema_forecast_weather,
        )
        df_new_historical_weather = pl.from_pandas(
            df_new_historical_weather[self.historical_weather_cols],
            schema_overrides=self.schema_historical_weather,
        )
        df_new_target = pl.from_pandas(
            df_new_target[self.target_cols], schema_overrides=self.schema_target
        )

        self.df_client = pl.concat([self.df_client, df_new_client]).unique(
            ["date", "county", "is_business", "product_type"]
        )
        self.df_gas_prices = pl.concat([self.df_gas_prices, df_new_gas_prices]).unique(
            ["forecast_date"]
        )
        self.df_electricity_prices = pl.concat(
            [self.df_electricity_prices, df_new_electricity_prices]
        ).unique(["forecast_date"])
        self.df_forecast_weather = pl.concat(
            [self.df_forecast_weather, df_new_forecast_weather]
        ).unique(["forecast_datetime", "latitude", "longitude", "hours_ahead"])
        self.df_historical_weather = pl.concat(
            [self.df_historical_weather, df_new_historical_weather]
        ).unique(["datetime", "latitude", "longitude"])
        self.df_target = pl.concat([self.df_target, df_new_target]).unique(
            ["datetime", "county", "is_business", "product_type", "is_consumption"]
        )

    def preprocess_test(self, df_test):
        df_test = df_test.rename(columns={"prediction_datetime": "datetime"})
        df_test = pl.from_pandas(
            df_test[self.data_cols[1:]], schema_overrides=self.schema_data
        )
        return df_test

In [28]:
class FeatureEngineer:
    def __init__(self, data):
        self.data = data
        self.estonian_holidays = list(
            holidays.country_holidays("EE", years=range(2021, 2026)).keys()
        )

    def _general_features(self, df_features):
        df_features = (
            df_features.with_columns(
                pl.col("datetime").dt.ordinal_day().alias("dayofyear"),pl.col("datetime").dt.hour().alias("hour"),
                pl.col("datetime").dt.day().alias("day"),pl.col("datetime").dt.weekday().alias("weekday"),
                pl.col("datetime").dt.month().alias("month"),pl.col("datetime").dt.year().alias("year"),
            )
            .with_columns(pl.concat_str("county","is_business","product_type","is_consumption",separator="_",).alias("segment"),)
            .with_columns(
                (np.pi * pl.col("dayofyear") / 183).sin().alias("sin(dayofyear)"),(np.pi * pl.col("dayofyear") / 183).cos().alias("cos(dayofyear)"),
                (np.pi * pl.col("hour") / 12).sin().alias("sin(hour)"),(np.pi * pl.col("hour") / 12).cos().alias("cos(hour)"),
            )
        )
        return df_features

    def _client_features(self, df_features):
        df_client = self.data.df_client

        df_features = df_features.join(
            df_client.with_columns((pl.col("date") + pl.duration(days=2)).cast(pl.Date)
            ),on=["county", "is_business", "product_type", "date"],how="left",
        )
        return df_features
    
    def is_country_holiday(self, row):
        return (
            datetime.date(row["year"], row["month"], row["day"])
            in self.estonian_holidays
        )

    def _holidays_features(self, df_features):
        df_features = df_features.with_columns(
            pl.struct(["year", "month", "day"])
            .apply(self.is_country_holiday)
            .alias("is_country_holiday")
        )
        return df_features

    def _forecast_weather_features(self, df_features):
        df_forecast_weather = self.data.df_forecast_weather
        df_weather_station_to_county_mapping = (
            self.data.df_weather_station_to_county_mapping
        )

        df_forecast_weather = (
            df_forecast_weather.rename({"forecast_datetime": "datetime"})
            .filter((pl.col("hours_ahead") >= 22) & pl.col("hours_ahead") <= 45)
#             .drop("hours_ahead")
            .with_columns(pl.col("latitude").cast(pl.datatypes.Float32),pl.col("longitude").cast(pl.datatypes.Float32),)
            .join(df_weather_station_to_county_mapping,how="left",on=["longitude", "latitude"],).drop("longitude", "latitude"))

        df_forecast_weather_date = (df_forecast_weather.group_by("datetime").mean().drop("county"))

        df_forecast_weather_local = (df_forecast_weather.filter(pl.col("county").is_not_null()).group_by("county", "datetime").mean())

        for hours_lag in [0, 7 * 24]:
            df_features = df_features.join(
                df_forecast_weather_date.with_columns(
                    pl.col("datetime") + pl.duration(hours=hours_lag)
                ),on="datetime",how="left",suffix=f"_forecast_{hours_lag}h",
            )
            df_features = df_features.join(
                df_forecast_weather_local.with_columns(
                    pl.col("datetime") + pl.duration(hours=hours_lag)
                ),on=["county", "datetime"],how="left",suffix=f"_forecast_local_{hours_lag}h",
            )

        return df_features

    def _historical_weather_features(self, df_features):
        df_historical_weather = self.data.df_historical_weather
        df_weather_station_to_county_mapping = (self.data.df_weather_station_to_county_mapping)

        df_historical_weather = (
            df_historical_weather.with_columns(pl.col("latitude").cast(pl.datatypes.Float32),pl.col("longitude").cast(pl.datatypes.Float32),
            ).join(df_weather_station_to_county_mapping,how="left",on=["longitude", "latitude"],).drop("longitude", "latitude")
        )

        df_historical_weather_date = (df_historical_weather.group_by("datetime").mean().drop("county"))

        df_historical_weather_local = (
            df_historical_weather.filter(pl.col("county").is_not_null()).group_by("county", "datetime").mean()
        )

        for hours_lag in [2 * 24, 7 * 24]:
            df_features = df_features.join(
                df_historical_weather_date.with_columns(
                    pl.col("datetime") + pl.duration(hours=hours_lag)
                ),on="datetime",how="left",suffix=f"_historical_{hours_lag}h",
            )
            df_features = df_features.join(
                df_historical_weather_local.with_columns(
                    pl.col("datetime") + pl.duration(hours=hours_lag)
                ),on=["county", "datetime"],how="left",suffix=f"_historical_local_{hours_lag}h",
            )

        for hours_lag in [1 * 24]:
            df_features = df_features.join(
                df_historical_weather_date.with_columns(
                    pl.col("datetime") + pl.duration(hours=hours_lag),
                    pl.col("datetime").dt.hour().alias("hour"),).filter(pl.col("hour") <= 10).drop("hour"),on="datetime",how="left",
                suffix=f"_historical_{hours_lag}h",
            )

        return df_features

    def _target_features(self, df_features):
        df_target = self.data.df_target

        df_target_all_type_sum = (df_target.group_by(["datetime", "county", "is_business", "is_consumption"]).sum().drop("product_type"))

        df_target_all_county_type_sum = (df_target.group_by(["datetime", "is_business", "is_consumption"]).sum().drop("product_type", "county"))
        
        hours_list=[i*24 for i in range(2,15)]

        for hours_lag in hours_list:
            df_features = df_features.join(
                df_target.with_columns(pl.col("datetime") + pl.duration(hours=hours_lag)
                ).rename({"target": f"target_{hours_lag}h"}),
                on=["county","is_business","product_type","is_consumption","datetime",],
                how="left",
            )

        for hours_lag in [2 * 24, 3 * 24, 7 * 24, 14 * 24]:
            df_features = df_features.join(
                df_target_all_type_sum.with_columns(pl.col("datetime") + pl.duration(hours=hours_lag)
                ).rename({"target": f"target_all_type_sum_{hours_lag}h"}),
                on=["county", "is_business", "is_consumption", "datetime"],how="left",
            )

            df_features = df_features.join(
                df_target_all_county_type_sum.with_columns(pl.col("datetime") + pl.duration(hours=hours_lag)).rename({"target": f"target_all_county_type_sum_{hours_lag}h"}),
                on=["is_business", "is_consumption", "datetime"],how="left",
                suffix=f"_all_county_type_sum_{hours_lag}h",
            )

        cols_for_stats = [f"target_{hours_lag}h" for hours_lag in hours_list[:4]]
        
        df_features = df_features.with_columns(
            df_features.select(cols_for_stats).mean(axis=1).alias(f"target_mean"),
            df_features.select(cols_for_stats).transpose().std().transpose().to_series().alias(f"target_std"),
            )

        for target_prefix, lag_nominator, lag_denomonator in [
            ("target", 24 * 7, 24 * 14),("target", 24 * 2, 24 * 9),("target", 24 * 3, 24 * 10),("target", 24 * 2, 24 * 3),
            ("target_all_type_sum", 24 * 2, 24 * 3),("target_all_type_sum", 24 * 7, 24 * 14),
            ("target_all_county_type_sum", 24 * 2, 24 * 3),("target_all_county_type_sum", 24 * 7, 24 * 14),
        ]:
            df_features = df_features.with_columns(
                (pl.col(f"{target_prefix}_{lag_nominator}h")/ (pl.col(f"{target_prefix}_{lag_denomonator}h") + 1e-3)
                ).alias(f"{target_prefix}_ratio_{lag_nominator}_{lag_denomonator}")
            )

        return df_features

    def _reduce_memory_usage(self, df_features):
        df_features = df_features.with_columns(pl.col(pl.Float64).cast(pl.Float32))
        return df_features

    def _drop_columns(self, df_features):
        df_features = df_features.drop("datetime", "hour", "dayofyear")
        return df_features

    def _to_pandas(self, df_features, y):
        cat_cols = ["county","is_business","product_type","is_consumption","segment",]

        if y is not None:
            df_features = pd.concat([df_features.to_pandas(), y.to_pandas()], axis=1)
        else:
            df_features = df_features.to_pandas()

        df_features = df_features.set_index("row_id")
        df_features[cat_cols] = df_features[cat_cols].astype("category")

        return df_features
    
    # added some new features here
    def _additional_features(self,df):
        for col in [
                    'temperature', 
                    'dewpoint', 
                    '10_metre_u_wind_component', 
                    '10_metre_v_wind_component', 
            ]:
            for window in [1]:
                df[f"{col}_diff_{window}"] = df.groupby(["county", 'is_consumption', 'product_type', 'is_business'])[col].diff(window)
        return df
    
    def _log_outliers(self,df):
        l1=['installed_capacity', 'target_mean', 'target_std']
        for i in l1:
            df = df.with_columns([(f"log_{i}", pl.when(df[i] != 0).then(np.log(pl.col(i))).otherwise(0))])
        return df
        

    def generate_features(self, df_prediction_items,isTrain):
        if "target" in df_prediction_items.columns:
            df_prediction_items, y = (
                df_prediction_items.drop("target"),
                df_prediction_items.select("target"),
            )
        else:
            y = None

        df_features = df_prediction_items.with_columns(
            pl.col("datetime").cast(pl.Date).alias("date"),
        )

        for add_features in [
            self._general_features,self._client_features,self._forecast_weather_features,
            self._historical_weather_features,self._target_features,self._holidays_features,
            self._log_outliers,self._reduce_memory_usage,self._drop_columns,]:
            df_features = add_features(df_features)

        df_features = self._to_pandas(df_features, y)
        df_features = self._additional_features(df_features)

        return df_features


In [29]:
class FeaturesGenerator:
    def __init__(self, data_storage):
        self.data_storage = data_storage
        self.estonian_holidays = list(
            holidays.country_holidays("EE", years=range(2021, 2026)).keys()
        )

    def _add_general_features(self, df_features):
        df_features = (
            df_features.with_columns(
                pl.col("datetime").dt.ordinal_day().alias("dayofyear"),
                pl.col("datetime").dt.hour().alias("hour"),
                pl.col("datetime").dt.day().alias("day"),
                pl.col("datetime").dt.weekday().alias("weekday"),
                pl.col("datetime").dt.month().alias("month"),
                pl.col("datetime").dt.year().alias("year"),
            )
            .with_columns(
                pl.concat_str(
                    "county",
                    "is_business",
                    "product_type",
                    "is_consumption",
                    separator="_",
                ).alias("segment"),
            )
            .with_columns(
                (np.pi * pl.col("dayofyear") / 183).sin().alias("sin(dayofyear)"),
                (np.pi * pl.col("dayofyear") / 183).cos().alias("cos(dayofyear)"),
                (np.pi * pl.col("hour") / 12).sin().alias("sin(hour)"),
                (np.pi * pl.col("hour") / 12).cos().alias("cos(hour)"),
            )
        )
        return df_features

    def _add_client_features(self, df_features):
        df_client = self.data_storage.df_client

        df_features = df_features.join(
            df_client.with_columns(
                (pl.col("date") + pl.duration(days=2)).cast(pl.Date)
            ),
            on=["county", "is_business", "product_type", "date"],
            how="left",
        )
        return df_features
    
    def is_country_holiday(self, row):
        return (
            datetime.date(row["year"], row["month"], row["day"])
            in self.estonian_holidays
        )

    def _add_holidays_features(self, df_features):
        df_features = df_features.with_columns(
            pl.struct(["year", "month", "day"])
            .apply(self.is_country_holiday)
            .alias("is_country_holiday")
        )
        return df_features

    def _add_forecast_weather_features(self, df_features):
        df_forecast_weather = self.data_storage.df_forecast_weather
        df_weather_station_to_county_mapping = (
            self.data_storage.df_weather_station_to_county_mapping
        )

        df_forecast_weather = (
            df_forecast_weather.rename({"forecast_datetime": "datetime"})
            .filter((pl.col("hours_ahead") >= 22) & pl.col("hours_ahead") <= 45)
            .drop("hours_ahead")
            .with_columns(
                pl.col("latitude").cast(pl.datatypes.Float32),
                pl.col("longitude").cast(pl.datatypes.Float32),
            )
            .join(
                df_weather_station_to_county_mapping,
                how="left",
                on=["longitude", "latitude"],
            )
            .drop("longitude", "latitude")
        )

        df_forecast_weather_date = (
            df_forecast_weather.group_by("datetime").mean().drop("county")
        )

        df_forecast_weather_local = (
            df_forecast_weather.filter(pl.col("county").is_not_null())
            .group_by("county", "datetime")
            .mean()
        )

        for hours_lag in [0, 7 * 24]:
            df_features = df_features.join(
                df_forecast_weather_date.with_columns(
                    pl.col("datetime") + pl.duration(hours=hours_lag)
                ),
                on="datetime",
                how="left",
                suffix=f"_forecast_{hours_lag}h",
            )
            df_features = df_features.join(
                df_forecast_weather_local.with_columns(
                    pl.col("datetime") + pl.duration(hours=hours_lag)
                ),
                on=["county", "datetime"],
                how="left",
                suffix=f"_forecast_local_{hours_lag}h",
            )

        return df_features

    def _add_historical_weather_features(self, df_features):
        df_historical_weather = self.data_storage.df_historical_weather
        df_weather_station_to_county_mapping = (
            self.data_storage.df_weather_station_to_county_mapping
        )

        df_historical_weather = (
            df_historical_weather.with_columns(
                pl.col("latitude").cast(pl.datatypes.Float32),
                pl.col("longitude").cast(pl.datatypes.Float32),
            )
            .join(
                df_weather_station_to_county_mapping,
                how="left",
                on=["longitude", "latitude"],
            )
            .drop("longitude", "latitude")
        )

        df_historical_weather_date = (
            df_historical_weather.group_by("datetime").mean().drop("county")
        )

        df_historical_weather_local = (
            df_historical_weather.filter(pl.col("county").is_not_null())
            .group_by("county", "datetime")
            .mean()
        )

        for hours_lag in [2 * 24, 7 * 24]:
            df_features = df_features.join(
                df_historical_weather_date.with_columns(
                    pl.col("datetime") + pl.duration(hours=hours_lag)
                ),
                on="datetime",
                how="left",
                suffix=f"_historical_{hours_lag}h",
            )
            df_features = df_features.join(
                df_historical_weather_local.with_columns(
                    pl.col("datetime") + pl.duration(hours=hours_lag)
                ),
                on=["county", "datetime"],
                how="left",
                suffix=f"_historical_local_{hours_lag}h",
            )

        for hours_lag in [1 * 24]:
            df_features = df_features.join(
                df_historical_weather_date.with_columns(
                    pl.col("datetime") + pl.duration(hours=hours_lag),
                    pl.col("datetime").dt.hour().alias("hour"),
                )
                .filter(pl.col("hour") <= 10)
                .drop("hour"),
                on="datetime",
                how="left",
                suffix=f"_historical_{hours_lag}h",
            )

        return df_features

    def _add_target_features(self, df_features):
        df_target = self.data_storage.df_target

        df_target_all_type_sum = (
            df_target.group_by(["datetime", "county", "is_business", "is_consumption"])
            .sum()
            .drop("product_type")
        )

        df_target_all_county_type_sum = (
            df_target.group_by(["datetime", "is_business", "is_consumption"])
            .sum()
            .drop("product_type", "county")
        )

        for hours_lag in [2 * 24,3 * 24,4 * 24,5 * 24,6 * 24,7 * 24,8 * 24,9 * 24,10 * 24,11 * 24,12 * 24,13 * 24,14 * 24,]:
            df_features = df_features.join(
                df_target.with_columns(
                    pl.col("datetime") + pl.duration(hours=hours_lag)
                ).rename({"target": f"target_{hours_lag}h"}),
                on=["county","is_business","product_type","is_consumption","datetime",],
                how="left",
            )

        for hours_lag in [2 * 24, 3 * 24, 7 * 24, 14 * 24]:
            df_features = df_features.join(
                df_target_all_type_sum.with_columns(
                    pl.col("datetime") + pl.duration(hours=hours_lag)
                ).rename({"target": f"target_all_type_sum_{hours_lag}h"}),
                on=["county", "is_business", "is_consumption", "datetime"],
                how="left",
            )

            df_features = df_features.join(
                df_target_all_county_type_sum.with_columns(
                    pl.col("datetime") + pl.duration(hours=hours_lag)
                ).rename({"target": f"target_all_county_type_sum_{hours_lag}h"}),
                on=["is_business", "is_consumption", "datetime"],
                how="left",
                suffix=f"_all_county_type_sum_{hours_lag}h",
            )

        cols_for_stats = [
            f"target_{hours_lag}h" for hours_lag in [2 * 24, 3 * 24, 4 * 24, 5 * 24]
        ]
        df_features = df_features.with_columns(
            df_features.select(cols_for_stats).mean(axis=1).alias(f"target_mean"),
            df_features.select(cols_for_stats)
            .transpose()
            .std()
            .transpose()
            .to_series()
            .alias(f"target_std"),
        )

        for target_prefix, lag_nominator, lag_denomonator in [
            ("target", 24 * 7, 24 * 14),
            ("target", 24 * 2, 24 * 9),
            ("target", 24 * 3, 24 * 10),
            ("target", 24 * 2, 24 * 3),
            ("target_all_type_sum", 24 * 2, 24 * 3),
            ("target_all_type_sum", 24 * 7, 24 * 14),
            ("target_all_county_type_sum", 24 * 2, 24 * 3),
            ("target_all_county_type_sum", 24 * 7, 24 * 14),
        ]:
            df_features = df_features.with_columns(
                (
                    pl.col(f"{target_prefix}_{lag_nominator}h")
                    / (pl.col(f"{target_prefix}_{lag_denomonator}h") + 1e-3)
                ).alias(f"{target_prefix}_ratio_{lag_nominator}_{lag_denomonator}")
            )

        return df_features

    def _reduce_memory_usage(self, df_features):
        df_features = df_features.with_columns(pl.col(pl.Float64).cast(pl.Float32))
        return df_features

    def _drop_columns(self, df_features):
        df_features = df_features.drop(
            "date", "datetime", "hour", "dayofyear"
        )
        return df_features

    def _to_pandas(self, df_features, y):
        cat_cols = [
            "county",
            "is_business",
            "product_type",
            "is_consumption",
            "segment",
        ]

        if y is not None:
            df_features = pd.concat([df_features.to_pandas(), y.to_pandas()], axis=1)
        else:
            df_features = df_features.to_pandas()

        df_features = df_features.set_index("row_id")
        df_features[cat_cols] = df_features[cat_cols].astype("category")

        return df_features
    
    def generate_features(self, df_prediction_items):
        if "target" in df_prediction_items.columns:
            df_prediction_items, y = (
                df_prediction_items.drop("target"),
                df_prediction_items.select("target"),
            )
        else:
            y = None

        df_features = df_prediction_items.with_columns(
            pl.col("datetime").cast(pl.Date).alias("date"),
        )

        for add_features in [
            self._add_general_features,
            self._add_client_features,
            self._add_forecast_weather_features,
            self._add_historical_weather_features,
            self._add_target_features,
            self._add_holidays_features,
            self._reduce_memory_usage,
            self._drop_columns,
        ]:
            df_features = add_features(df_features)

        df_features = self._to_pandas(df_features, y)

        return df_features

## initialize

In [30]:
data_storage = DataStorage()
features_generator = FeaturesGenerator(data_storage=data_storage)
feat_gen = FeatureEngineer(data=data_storage)

In [ ]:
df_train_features = features_generator.generate_features(data_storage.df_data)
df_train_features = df_train_features[df_train_features['target'].notnull()]

df_train = feat_gen.generate_features(data_storage.df_data,True)
df_train = df_train[df_train['target'].notnull()]

In [17]:
env = enefit.make_env()
iter_test = env.iter_test()

In [18]:
for (
    df_test, 
    df_new_target, 
    df_new_client, 
    df_new_historical_weather,
    df_new_forecast_weather, 
    df_new_electricity_prices, 
    df_new_gas_prices, 
    df_sample_prediction
) in iter_test:

    data_storage.update_with_new_data(
        df_new_client=df_new_client,
        df_new_gas_prices=df_new_gas_prices,
        df_new_electricity_prices=df_new_electricity_prices,
        df_new_forecast_weather=df_new_forecast_weather,
        df_new_historical_weather=df_new_historical_weather,
        df_new_target=df_new_target
    )
    
    #separately generate test features for both models
    
    df_test = data_storage.preprocess_test(df_test)
    
    df_test_features = features_generator.generate_features(df_test)
    
    df_test_feats = feat_gen.generate_features(df_test,False)
    
    df_test_feats.drop(columns=['date','literal'],inplace=True)
        
    pred1 = predict(df_test_features)
    
    pred2 = predict_model(df_test_feats)
    
    # Ensembling with slightly tuned model weights
    pred_1_w = 0.49
    df_sample_prediction["target"] = (
        (pred_1_w * pred1) + 
        ((1 - pred_1_w) * pred2)
    )
    
    env.predict(df_sample_prediction)
    gc.collect()

TypeError: from_pandas() got an unexpected keyword argument 'strict'

In [ ]:
preds = pd.read_csv(os.path.join("example_test_files", "revealed_targets.csv"))
p = preds[['row_id', 'data_block_id', 'target']]
p['row_id'] = p['row_id'] + 6336

In [ ]:
for X_init, y_init in iter_test:
    print(env._status)
    current_data_block_id = X_init.data_block_id.drop_duplicates().to_list()[0]
    p_current = p[p.data_block_id == current_data_block_id]
    # print(X_init)
    # print(y_init)
    # print(p_current)
    
    env.predict(p_current)
    print(env._status)
    env.iter_test()



In [ ]:
env._status

In [ ]:
import json
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import sklearn

In [ ]:
import geopandas as gpd

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

In [ ]:
import datetime
import pytz
import holidays
import optuna
import joblib

In [ ]:
from typing import Optional,Dict,Tuple
from pathlib import Path

In [ ]:
from xgboost import XGBRegressor

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('.'))

if module_path not in sys.path:
    sys.path.append(module_path)
    
from solar_tracking import calculate_elevation_angle, calculate_irradiation_on_surface

In [ ]:
os.chdir('../')

plt.style.use("ggplot")
plt.rcParams.update(**{'figure.dpi': 150})

### Load Files

In [ ]:
ESTONIA_GPKG = "estonia.gpkg"

# Coordinate Reference System (CRS)
ESTONIA_CRS = "EPSG:4133"

In [ ]:
# Read in all spatial layers
# boundry = gpd.read_file(ESTONIA_GPKG, layer='boundry', driver="GPKG", crs=ESTONIA_CRS)
counties = gpd.read_file(os.path.join(os.getcwd(), ESTONIA_GPKG), layer='counties', driver="GPKG", crs=ESTONIA_CRS)
# municipalities = gpd.read_file(ESTONIA_GPKG, layer='municipalities', driver="GPKG", crs=ESTONIA_CRS)
# settlements = gpd.read_file(ESTONIA_GPKG, layer='settlements', driver="GPKG", crs=ESTONIA_CRS)

In [ ]:
# Plot the counties of Estonia as well as the centroids to those counties
fig, ax = plt.subplots(figsize=(6,8))
counties.plot(ax=ax)
counties.centroid.plot(markersize=12,color='red',ax=ax)
plt.axis('off')
plt.tight_layout();

In [ ]:
weather = pd.read_csv('historical_weather.csv', parse_dates=['datetime'])
forecast = pd.read_csv('forecast_weather.csv', parse_dates=['origin_datetime', 'forecast_datetime'])

In [ ]:
weather['utc_offset'] = weather.datetime.apply(handle_timezone)

In [ ]:
forecast['utc_offset'] = forecast.origin_datetime.apply(handle_timezone)

In [ ]:
weather_gdf = gpd.GeoDataFrame(
    weather, geometry=gpd.points_from_xy(weather.longitude, weather.latitude), crs=ESTONIA_CRS
)

In [ ]:
forecast_gdf = gpd.GeoDataFrame(
    forecast, geometry=gpd.points_from_xy(forecast.longitude, forecast.latitude), crs=ESTONIA_CRS
)

In [ ]:
county_data = gpd.overlay(counties,\
                          weather_gdf,\
                          how='intersection',\
                          keep_geom_type=False\
)

In [ ]:
county_pred = gpd.overlay(counties,\
                          forecast_gdf,\
                          how='intersection',\
                          keep_geom_type=False\
)

In [ ]:
solar_min = weather[weather.direct_solar_radiation > 0].direct_solar_radiation.min()

In [ ]:
solar_max = weather.direct_solar_radiation.max()

In [ ]:
train_df = pd.read_csv('train.csv')

In [ ]:
earliest_time = train_df['datetime'].min()
latest_time = train_df['datetime'].max()

In [ ]:
earliest_time

In [ ]:
def max_p(values, k):
    """
    Given a list of values and `k` bins,
    returns a list of their Maximum P bin number.
    """
    binning = MaxP(values, k=k)
    return binning.yb


def bin_data(values, k, interval="equal"):
    """
    Given a list of values, 'k' bin
    return categorical bin group ids in specified intervals
    """
    min_val = values.min()
    max_val = values.max()
    
    binned_data = []
    thresholds = []
    
    if interval == "equal":
        intv = (max_val - min_val) / k
        thresholds = [i*intv for i in range(k)]
        
    elif interval == "stdev":
        intv = values.std()
        avg = values.mean()
        if k % 2 == 0:
            k_half = int(k/2)
            k_res = 0
        else:
            k_half = int((k-1)/2)
            k_res = 0.5
            
        for i in range(-1*k_half, k_half+1):
            if i == 0 and k_res == 0:
                pass
            else:
                thresholds.append((i*intv) + avg)

    elif interval == "equal_count":
        pass
        
    elif interval == "quantile":
        pass
    
    elif interval == "log_scale":
        pass

    else:
        pass
    

    binned_data = []
    for val in values:
        for j in range(0, len(thresholds)):
            if val < thresholds[j]:
                binned_data.append(j)
            else:
                pass
    
    return pd.Series(binned_data)
    
    


def extract_dt_attributes(df: pd.DataFrame):
    # convert datetime column, if not done already
    df['datetime'] = pd.to_datetime(df['datetime'])
    
    # dates and times
    df['date'] = df['datetime'].dt.date
    df['time'] = df['datetime'].dt.strftime('%H:%M:%S')
    
    #
    df['year'] = df['datetime'].dt.year
    df['day_of_year'] = df['datetime'].dt.strftime('%j').astype(int)
    df['datediff_in_days'] = (
        df['datetime']- earliest_time
    ).dt.days
    
    # dictionary with time features as keys
    # and min and max as values
    time_features = {
        'hour': [0, 23],
        'dayofweek': [0, 6],
        'week': [1, 52],
        'month': [1, 12]
    }
    
    for col in time_features:
        if col=='week':
            df[col] = df['datetime'].dt.isocalendar().week.astype(np.int32)
        else:
            df[col] = getattr(df['datetime'].dt,col)
        
        
        ## sin and cosine features to capture the circular continuity
        col_min,col_max = time_features[col]
        angles = 2*np.pi*(df[col]-col_min)/(col_max-col_min+1)
        
        # add sin and cos
        df[col+'_sine'] = np.sin(angles).astype('float')
        df[col+'_cosine'] = np.cos(angles).astype('float')
    return df

In [ ]:
def handle_timezone(dt):
    """
    https://www.timeanddate.com/time/change/estonia/tallinn
    """
    year = dt.year
    
    if year == 2021:
        
        if dt >= datetime.datetime.strptime("2021-10-31 03:00:00", '%Y-%m-%d %H:%M:%S'):
            utc_offset = 2
        else:
            utc_offset = 3
        
        
    elif year == 2022:
        
        if dt < datetime.datetime.strptime("2022-03-27 03:00:00", '%Y-%m-%d %H:%M:%S'):
            utc_offset = 2
        elif dt >= datetime.datetime.strptime("2022-10-30 03:00:00", '%Y-%m-%d %H:%M:%S'):
            utc_offset = 2
        else:
            utc_offset = 3
        
        
        
    elif year == 2023:
        
        if dt < datetime.datetime.strptime("2023-03-26 03:00:00", '%Y-%m-%d %H:%M:%S'):
            utc_offset = 2
            
        elif dt >= datetime.datetime.strptime("2023-10-29 03:00:00", '%Y-%m-%d %H:%M:%S'):
            utc_offset = 2
        
        else:
            utc_offset = 3
        
        
    else:
        pass
    
    return utc_offset
    

In [ ]:
train_df

In [ ]:
extract_dt_attributes(county_data)

In [ ]:
# this_month = 1
# monthly_avg = county_data[(county_data.month == this_month) & (county_data.direct_solar_radiation > 0.0)].groupby(by=['countyCodes']).mean()
# monthly_max = county_data[(county_data.month == this_month) & (county_data.direct_solar_radiation > 0.0)].groupby(by=['countyCodes']).max()

In [ ]:
# monthly_avg = pd.merge(monthly_avg, counties, on='countyCodes')
# monthly_max = pd.merge(monthly_max, counties, on='countyCodes')

In [ ]:
# county_data['direct_solar_bin'] = bin_data(county_data['direct_solar_radiation'], k=6, interval="stdev")

In [ ]:
# for month in range(1, 13):

#     monthly_avg = county_data[(county_data.month == month) & (county_data.direct_solar_radiation > 0.0)].groupby(by=['countyCodes']).median()
#     monthly_max = county_data[(county_data.month == month) & (county_data.direct_solar_radiation > 0.0)].groupby(by=['countyCodes']).max()
#     monthly_avg = pd.merge(monthly_avg.reset_index(), counties, on='countyCodes')
#     monthly_max = pd.merge(monthly_max.reset_index(), counties, on='countyCodes')

#     fig, ax = plt.subplots(2, 1, sharex=True, sharey=True)

#     divider_0 = make_axes_locatable(ax[0])
#     divider_1 = make_axes_locatable(ax[1])

#     cax_0 = divider_0.append_axes("top", size="5%", pad=0.5)
#     cax_1 = divider_1.append_axes("bottom", size="5%", pad=0.5)

#     monthly_avg.plot(
#         column="direct_solar_radiation",
#         ax=ax[0],
#         legend=True,
#         cax=cax_0,
#         legend_kwds={"label": f"Nom Direct Solar Radiation: Month {month}", "orientation": "horizontal"},
#         cmap='OrRd'
#     );
#     monthly_max.plot(
#         column="direct_solar_radiation",
#         ax=ax[1],
#         legend=True,
#         cax=cax_1,
#         legend_kwds={"label": f"Max Direct Solar Radiation: Month {month}", "orientation": "horizontal"}
#     );
    
    
#     print("\n\n\n")

In [ ]:
# forecast[forecast.hours_ahead == 1]

In [ ]:
station = pd.read_csv('weather_station_to_county_mapping.csv')

In [ ]:
# station.dropna()

In [ ]:
with open('county_id_to_name_map.json', 'r') as fw:
    county_lookup = json.load(fw)

### Solar Production Analysis

In [ ]:
spa = train_df[train_df['is_consumption'] == 0]
spa = spa.rename(columns={'county': 'countyCodes'})
spa['countyCodes'] = spa['countyCodes'].astype(str)
spa = spa.set_index('countyCodes', 'datetime')
county_data['countyCodes'] = county_data['countyCodes'].astype(str)
county_weather_data = county_data.set_index('countyCodes', 'datetime')
spa = pd.merge(spa, county_weather_data, on=['countyCodes', 'datetime'])

In [ ]:
filter_columns_spa = ['countyCodes', 'datetime', 'is_business', 'product_type', 'prediction_unit_id', 'target', 
                      'direct_solar_radiation', 'diffuse_radiation', 'shortwave_radiation',
                      'temperature', 'dewpoint', 'rain', 'snowfall', 'surface_pressure', 'cloudcover_total',
                      'cloudcover_low', 'cloudcover_mid', 'cloudcover_high',
                      'dayofweek', 'dayofweek_sine', 'dayofweek_cosine', 'week', 'week_sine', 'week_cosine',
                      'month', 'month_sine', 'month_cosine', 'day_of_year', 'latitude', 'longitude']

x_columns = [col for col in filter_columns_spa if col != "target"]
y_columns = ["target"]
spa = spa.reset_index()
spa = spa[filter_columns_spa]
spa

In [ ]:
spa['elevation'] = spa[['datetime', 'day_of_year', 'longitude', 'latitude']].apply(
    lambda x: calculate_elevation_angle(
        local_time=x[0], day_of_year=x[1], longitude=x[2], latitude=x[3], utc_offset=2
    ), axis=1
)

In [ ]:
spa.product_type.value_counts()

In [ ]:
X = spa[x_columns]
del X['datetime']
y = spa[y_columns]

In [ ]:
X_business = X[X.is_business == 1]
X_consumer = X[X.is_business == 0]

In [ ]:
X_business.product_type.value_counts()

In [ ]:
X_consumer.product_type.value_counts()

In [ ]:
category_columns = ['countyCodes', 'is_business', 'product_type', 'prediction_unit_id']


for col in category_columns:
    X[col] = X[col].astype('category')
    del X[col]
X.dtypes

In [ ]:
y = y.fillna(0)

In [ ]:
def fit_model(
    X:pd.DataFrame,
    y:pd.Series,
    config:Optional[Dict]=None,
    n_jobs:int=1,
) -> XGBRegressor:
    '''
    Train a xgboost regressor with L1 loss
    '''
    model = XGBRegressor(
        objective='reg:absoluteerror',
        tree_method='hist',
        n_jobs=n_jobs,
        enable_categorical=True,
        base_score=y.mean()
    )
    
    if config:
        # if config is supplied, set the model hyperparameters
        model.set_params(**config)
        
    
    return model.fit(X,y)

In [ ]:
model_1 = XGBRegressor()
# model_1.enable_categorical = True
model_1.fit(X, y)

In [ ]:
test_ndx = 3593933
y_hat = []
for ndx in X.index:
    test_row = np.asarray([X.iloc[ndx]])
    pred_val = model_1.predict(test_row)
    y_hat.append(pred_val)
    
# actual_val = y.iloc[test_ndx]

# make a prediction
# yhat = model_1.predict(test_row)

# summarize prediction
# print('Predicted: %.3f' % yhat)
# print('Actual: %.3f' % actual_val)

In [ ]:
times = spa.datetime.to_list()
plt.plot(times, y.target.to_list(), 'b', linewidth=0.3)
plt.plot(times, y_hat, 'o', linewidth=0.3)

In [ ]:
# counts for the country,store,product
desc_columns = ['county','is_business','product_type','is_consumption']

fig, axs = plt.subplots(1, len(desc_columns), figsize=(5*len(desc_columns), 3))

for i, column in enumerate(desc_columns):
    _ = sns.countplot(train_df, x=column, ax=axs[i])

_ = fig.tight_layout()

In [ ]:
train_avgd = (
    train_df
    .groupby(['datetime','is_consumption'])
    ['target'].mean()
    .unstack()
    .rename({0: 'produced', 1:'consumed'}, axis=1)
)

fig, ax = plt.subplots(1, 1, figsize=(12, 4))
_ = train_avgd.plot(ax=ax, alpha=0.5)
_ = ax.set_ylabel('Energy consumed / produced')

In [ ]:
# plot of average weekly sales
fig,ax = plt.subplots(1,1,figsize=(6,4))
_ = train_avgd.resample('M').mean().plot(ax=ax, marker='.')
_ = ax.set_ylabel('Average monthly')

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(6,4))
train_avgd.groupby(train_avgd.index.hour).mean().plot(ax=ax, marker='.')
_ = ax.set_xlabel('Hour')

In [ ]:
# get train attributes
extract_dt_attributes(train_df)

In [ ]:
categorical_cols = ['county', 'product_type']
for column in categorical_cols:
    train_df[column] = train_df[column].astype('category')

In [ ]:
train_df[train_df['is_consumption'] == 0][['target', 'hour', 'month']].groupby(by=['month', 'hour']).mean()

In [ ]:
electricity_df = pd.read_csv('electricity_prices.csv')
gas_df = pd.read_csv('gas_prices.csv')
client_df = pd.read_csv('client.csv')

In [ ]:
client_df[(client_df.data_block_id == 4) & (client_df.county == 0)]

In [ ]:
train_df[(train_df.data_block_id == 1) & (train_df.county == 0)]

In [ ]:
print(f"Total Observed Hours: {len(train_df.datetime.drop_duplicates().to_list())}")
print(f"Counties: {len(train_df.county.drop_duplicates().to_list())}")
print(f"Prediction Types: {len(train_df.prediction_unit_id.drop_duplicates().to_list())}")
print(f"Data Blocks: {len(train_df.data_block_id.drop_duplicates().to_list())}")

<h4>Business</h4|>

In [ ]:
for county_id in train_df.county.drop_duplicates().to_list()[0:1]:
    county_df = train_df[train_df.county == county_id]
    for product_type in county_df.product_type.drop_duplicates().to_list()[0:1]:
        cpt_df = county_df[county_df.product_type == product_type]
        consumed_df = cpt_df[(cpt_df.is_business == 1) & (cpt_df.is_consumption == 1)]
        print(consumed_df.sort_values('datetime', ascending=True))
        x = range(len(consumed_df))
        y = consumed_df.target.to_list()
        plt.plot(x, y)
        plt.show()



<h4>Consumer</h4>

In [ ]:
for county_id in train_df.county.drop_duplicates().to_list()[0:1]:
    county_df = train_df[train_df.county == county_id]
    for product_type in county_df.product_type.drop_duplicates().to_list():
        cpt_df = county_df[county_df.product_type == product_type]
        non_biz_cpt_df = cpt_df[cpt_df.is_business == 0]
        print(non_biz_cpt_df)